# 012.b_SQL_Analysis_Apps_For_Book_Lovers

<a id='back'><a/>
# Table of Contents
    
- [1. SQL Analysis Apps For Book Lovers Overview](#1)
- [2. General Information Overview](#2)
- [3. SQL Queries](#3)
    - [3.1 Number of Books Released after January 1, 2000](#3.1)
    - [3.2 Number of User Reviews and Average Rating for Each Book](#3.2)
    - [3.3 Publisher that has Released the Greatest Number of Books of > 50 Pages](#3.3)
    - [3.4 Author with the Highest Average Book Rating (Min 50 Ravtings)](#3.4)
    - [3.5 Average Number of Text Reviews Among Users Who Rated > 50 Books](#3.5)

<a id='1'><a/>
# 1. SQL Analysis Apps For Book Lovers Overview
[Return to Table of Contents](#back)

Covid-19 took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. This attracted the attention of startups that rushed to develop new apps for book lovers.
We've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

<a id='2'><a/>
# 2. General Information Overview
[Return to Table of Contents](#back)

In [5]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [6]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [10]:
table_list = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table_name in table_list:
    query_tamplate = "SELECT * FROM {} LIMIT 5".format(table_name)
    print('Table name:', table_name)
    display(pd.io.sql.read_sql(query_tamplate, con = engine))

Table name: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Table name: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Table name: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Table name: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Table name: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<a id='3'><a/>
# 3. SQL Queries
[Return to Table of Contents](#back)

<a id='3.1'><a/>
## 3.1 Number of books released after January 1, 2000
[Return to Table of Contents](#back)

In [24]:
query2 = "SELECT COUNT(*) AS cnt FROM books WHERE publication_date > '2000-01-01'"

In [25]:
pd.io.sql.read_sql(query2, con = engine)

,cnt
0,819


There were 819 books released after January 1, 2000.

<a id='3.2'><a/>
## 3.2 Number of user reviews and average rating for each book
[Return to Table of Contents](#back)

In [28]:
query3 = '''
SELECT 
    ratings.book_id AS book_id,
    COUNT(DISTINCT review_id) AS review_id_uniq_cnt,
    AVG(rating) AS average 
FROM 
    reviews
    INNER JOIN ratings ON ratings.book_id = reviews.book_id 
GROUP BY 
    ratings.book_id 
'''

In [29]:
pd.io.sql.read_sql(query3, con = engine)

,book_id,review_id_uniq_cnt,average
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
989,996,3,3.666667
990,997,3,3.400000
991,998,4,3.200000
992,999,2,4.500000


<a id='3.3'><a/>
## 3.3 Publisher that has released the greatest number of books of > 50 pages
[Return to Table of Contents](#back)

In [30]:
query4 = "SELECT publisher_id, COUNT(book_id) AS cnt FROM books WHERE num_pages > 50 GROUP BY publisher_id ORDER BY cnt DESC"

In [31]:
pd.io.sql.read_sql(query4, con = engine).head(1)

,publisher_id,cnt
0,212,42


Publisher 212 released 42 books over 50 pages, the most of any other publisher.

In [32]:
query5 = '''
SELECT
    *
FROM
    publishers
WHERE
    publisher_id IN ('212');  
'''

In [33]:
pd.io.sql.read_sql(query5, con = engine)

,publisher_id,publisher
0,212,Penguin Books


<a id='3.4'><a/>
## 3.4 Author with the highest average book rating (min 50 ratings)
[Return to Table of Contents](#back)

In [34]:
query6 = '''
SELECT 
    author, 
    AVG(rating) AS average 
FROM 
    books 
    INNER JOIN authors ON authors.author_id = books.author_id 
    INNER JOIN ratings ON ratings.book_id = books.book_id 
GROUP BY 
    author 
HAVING COUNT(rating) > 49 
ORDER BY 
    average DESC 
LIMIT 5'''


In [35]:
pd.io.sql.read_sql(query6, con = engine).head(1)

,author,average
0,Diana Gabaldon,4.3


The author with the highest average book rating (considering a minimum of 50 ratings) is Diana Gabaldon.

<a id='3.5'><a/>
## 3.5 Average number of text reviews among users who rated > 50 books
[Return to Table of Contents](#back)

In [13]:
query7 = '''
SELECT 
    COUNT(*) / COUNT(distinct username) as average 
FROM 
    reviews 
WHERE 
    username 
IN 
    (SELECT username FROM ratings 
        GROUP BY 
            username 
        HAVING COUNT(rating) > 50)'''

In [14]:
pd.io.sql.read_sql(query7, con = engine)

,average
0,24


The average number of reviews was 24.